In [36]:
import numpy as np
import os
import time
import pandas as pd
import re
import json
import keras
import glob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
# Build the corpus and sequences
labelToName = { 0 : 'PREMISE', 1 : 'CONCLUSION' }
namesInLabelOrder = ['PREMISE', 'CONCLUSION']
#os.chdir('E:\git\SmartLawML\work-area')
#df=pd.read_csv('preprocessed.csv')

#X=df.text.tolist()
#labels=df.loc[:,['label']]

In [38]:
train_files = glob.glob("E:/git/SmartLawDataset/json/*.json")

mergejson=pd.DataFrame()
for file in train_files:
    #print(file)
    with open(file,encoding="utf8") as f1: 
        data1 = json.load(f1)
    df1=pd.DataFrame([data1])
    mergejson=pd.concat([mergejson,df1])
mg1=mergejson.drop(['header','background','order','footer','annotationProcessingStage','annotationProcessingStageAnnotations','processedText'], axis=1)
X=[]
labels=[]
for j in mg1['arguments']:
        for k in j:           
            for p in k['argumentSentences']:                
                if(p['argumentSentenceType']=='NA'):
                    continue
                X.append(p['text'])
                labels.append(p['argumentSentenceType'])
                

In [39]:
print(X[0])

[2] Perused the application and say filed by the I.O. Heard the Ld. Counsel for the Applicant and Ld. APP for the State.


In [40]:
def preprocess(temp):
    temp=re.sub("\s\s+"," ",temp)    #Replacing multiple spaces with one
    temp=temp.replace(u'\xa0', ' ').encode('utf-8')  #Replacing non-breaking space    
    temp=temp.decode().split(' ')    #Converting a string into list of words seperated by space charcter
    return temp

#print(X[1])
X = [preprocess(t) for t in X]  
#print("\n\n\n After Preprocess:\n\n",X[1])


In [41]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1).split(X, labels)
train_indices, test_indices = next(sss)

def seqlengths(l):
    if isinstance(l,list):
        yield len(l)
        for y in l:
            yield from seqlengths(y)

maxsequenceLength=max(seqlengths(X))
print("Max sequence length:",maxsequenceLength)

Max sequence length: 3076


In [42]:
# Encode the documents
kTokenizer = keras.preprocessing.text.Tokenizer() 
kTokenizer.fit_on_texts(X)
Xencoded = np.array([np.array(xi) for xi in kTokenizer.texts_to_sequences(X)])                

print("Shape Before Padding:",np.shape(Xencoded[22]))
Xencoded=keras.preprocessing.sequence.pad_sequences(Xencoded, maxlen=maxsequenceLength, truncating='post')
print("\nShape After Padding:",np.shape(Xencoded[22]))

labels = np.array(labels)

nWords=len(kTokenizer.word_index)
print("\n nWords:",nWords)

Shape Before Padding: (22,)

Shape After Padding: (3076,)

 nWords: 5051


C:\Users\abc\AppData\Local\Temp\ipykernel_4528\199922491.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Xencoded = np.array([np.array(xi) for xi in kTokenizer.texts_to_sequences(X)])


In [44]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=1).fit(X)
Xencoded=vectorizer.transform(X)

from sklearn.linear_model import LogisticRegression
results = {}
model = LogisticRegression()
train_x = Xencoded[train_indices]
test_x = Xencoded[test_indices]
train_labels = labels[train_indices]
test_labels = labels[test_indices]
model.fit(train_x, train_labels)
predicted_labels = model.predict(test_x)
results['confusion_matrix'] = confusion_matrix(labels[test_indices], predicted_labels).tolist()
results['classification_report'] = classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder, output_dict=True)
print ("\n\nConfusion Matrix:\n",confusion_matrix(labels[test_indices], predicted_labels))
print ("\n\nClassification Report:\n\n",classification_report(labels[test_indices], predicted_labels, digits=4, target_names=namesInLabelOrder))





Confusion Matrix:
 [[ 55  33]
 [  3 525]]


Classification Report:

               precision    recall  f1-score   support

     PREMISE     0.9483    0.6250    0.7534        88
  CONCLUSION     0.9409    0.9943    0.9669       528

    accuracy                         0.9416       616
   macro avg     0.9446    0.8097    0.8601       616
weighted avg     0.9419    0.9416    0.9364       616

